### ISDA SIMM homogenity and differentiability

The purpose of this notebook is to show case under which circumstance ISDA SIMM is a homogeneous and differentiable function.

Structure:

1. Discussion what characterizes a homogeneous function. Should be part of the Latex section on Euler allocation and not in this notebook.
2.

In [1]:
# import cell
from numpy import arange
from collateralAgreement.collateralAgreement import CollateralAgreement

from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import exportPlotlyFigure
from margining.simm import SIMM
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection
from marketdata import init_marketdata
import QuantLib as ql
import pandas as pd
import plotly.express as px

We create an USD IRS whose SIMM sensitivity is roughly at 2\3rd of the ISDA SIMM Liq threshold which is 230Mn USD.

In [2]:
notional = 200000000000

irs = IRS(notional = notional,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          swapDirection=SwapDirection.PAYER,
          index = InterestRateIndex.USDLIBOR3M)

irs_off_market = IRS(notional = notional,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          swapDirection=SwapDirection.PAYER,
          index = InterestRateIndex.USDLIBOR3M,
          fixed_rate= 0.02)

simm_sensis = irs.get_simm_sensis()

sum([float(entry['amountUSD']) for entry in simm_sensis])

177293785.5630342

We create a collateral agreement with associated ISDA SIMM and SA CCR model and load the irs in the portfolio.

In [3]:
ca = CollateralAgreement(mta = 1000000000,
                         threshold= 2000000000)
ca.link_sa_ccr_instance(SA_CCR(collateralAgreement=ca))
ca.add_trades(irs)

ca2 = CollateralAgreement(mta = 1000000000,
                         threshold= 2000000000)
ca2.link_sa_ccr_instance(SA_CCR(collateralAgreement=ca2))
ca2.add_trades(irs_off_market)

And we query the model for initial margin receive calculated and collateral received

In [4]:
print('IM receive is %f.2 USD' %ca.get_im_model().get_risk_measure())
print('VM balance is %f.2 USD' %ca.get_vm_model().get_risk_measure())
print('Collateral received is %f.2 USD' %ca.get_C())

IM receive is 9038157077.781643.2 USD
VM balance is 0.233376.2 USD
Collateral received is 9038157078.015018.2 USD


We set this as the models starting collateral amount

In [5]:
irs.get_fixed_rate()

0.024092650177355537

In [6]:
ca.set_start_collateral_amount(ca.get_C())
ca2.set_start_collateral_amount(ca2.get_C())
asdf =1

In [7]:
bumps = arange(0,2.01,0.01)
resultDataframe = pd.DataFrame(columns = ['USD','k','Legend'])

In [ ]:
def bump_and_get_results(bumpsize, trade, collateralagreement):
    record = {}
    record['Bumpsize']=bumpsize
    bumped_copy = trade.get_bumped_copy(rel_bump_size=bumpsize-1)
    collateralagreement.remove_all_trades()
    collateralagreement.add_trades(bumped_copy)
    record['IM'] = collateralagreement.get_im_model().get_risk_measure()
    record['VM'] = collateralagreement.get_vm_model().get_risk_measure()
    record['Collateral'] = collateralagreement.get_C()
    collateralagreement.remove_all_trades()
    collateralagreement.add_trades(trade)
    return record

for bump in bumps:
    result = bump_and_get_results(bump, irs, ca)
    im_record = {'X': result['Bumpsize'],
                 'Y': result['IM'],
                 'Legend': 'IM'}
    im_shadow_record = {'X': result['Bumpsize'],
                 'Y': result['IM'],
                 'Legend': 'calculated IM at the money IRS'}
    vm_record = {'X': result['Bumpsize'],
                 'Y': result['VM'],
                 'Legend': 'calculated VM at the money IRS'}
    resultDataframe = resultDataframe.append(im_record, ignore_index=True)
    resultDataframe = resultDataframe.append(im_shadow_record, ignore_index=True)
    resultDataframe = resultDataframe.append(vm_record, ignore_index=True)
    collateral_record = {'X': result['Bumpsize'],
                 'Y': result['Collateral'],
                 'Legend': 'C at par IRS'}
    resultDataframe = resultDataframe.append(collateral_record, ignore_index=True)

    result2 = bump_and_get_results(bump, irs_off_market, ca2)
    collateral_record2 = {'X': result2['Bumpsize'],
                 'Y': result2['Collateral'],
                 'Legend': 'C in the money IRS'}
    im_record2 = {'X': result2['Bumpsize'],
                 'Y': result2['IM'],
                 'Legend': 'calculated IM in the money IRS'}
    vm_record2 = {'X': result2['Bumpsize'],
                 'Y': result2['VM'],
                 'Legend': 'calculated VM in the money IRS'}
    resultDataframe = resultDataframe.append(collateral_record2, ignore_index=True)
    resultDataframe = resultDataframe.append(im_record2, ignore_index=True)
    resultDataframe = resultDataframe.append(vm_record2, ignore_index=True)

In [ ]:
fig = px.line(resultDataframe[resultDataframe.Legend == 'IM'], x='X', y='Y', color='Legend', line_dash='Legend')
fig.update_layout(xaxis_title = 'a', yaxis_title = 'USD')
fig.show()
exportPlotlyFigure(fig, name='ISDA_SIMM_homogenity')

In [ ]:
fig = px.line(resultDataframe[~resultDataframe.Legend.isin(['IM', 'calculated IM at the money IRS', 'calculated VM at the money IRS'])], x='X', y='Y', color='Legend', line_dash='Legend')
fig.update_layout(xaxis_title = 'a', yaxis_title = 'USD')
fig.show()
exportPlotlyFigure(fig, name='C_homogenity')